In [1]:
from Process import PlateReader
import pandas as pd
import matplotlib.pyplot as plt

### Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [2]:
sample_info = pd.read_csv(r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Temp or Working\07_08_21_Octadecene_Info_pt2")
sample_info

,UID,Labware,Slot,Well,ODE concentration wtf,SDS concentration wtf,ethanol concentration wtf,water concentration wtf,ODE amount mass g,ODE amount volume uL,...,SDS amount volume uL,ethanol amount mass g,ethanol amount volume uL,water amount mass g,water amount volume uL,ODE-ethanol-stock amount volume uL,SDS-ethanol-stock amount volume uL,ethanol-stock amount volume uL,water-stock amount volume uL,Total Volume
0,S2_A1_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A1,0.000000,0.0005,0.3,0.699500,0.000000,0.000000,...,0.594059,0.36,456.100342,0.839400,839.400000,0.000000,60.0,396.867824,839.400000,1296.267824
1,S2_A2_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A2,0.000000,0.0005,0.5,0.499500,0.000000,0.000000,...,0.594059,0.60,760.167237,0.599400,599.400000,0.000000,60.0,701.050333,599.400000,1360.450333
2,S2_A3_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A3,0.000000,0.0005,0.7,0.299500,0.000000,0.000000,...,0.594059,0.84,1064.234132,0.359400,359.400000,0.000000,60.0,1005.232843,359.400000,1424.632843
3,S2_A4_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A4,0.000111,0.0005,0.2,0.799389,0.000133,0.168990,...,0.594059,0.24,304.066895,0.959267,959.266667,26.666667,60.0,218.278893,959.266667,1264.212226
4,S2_A5_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A5,0.000111,0.0005,0.4,0.599389,0.000133,0.168990,...,0.594059,0.48,608.133789,0.719267,719.266667,26.666667,60.0,522.461402,719.266667,1328.394735
5,S2_A6_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A6,0.000111,0.0005,0.6,0.399389,0.000133,0.168990,...,0.594059,0.72,912.200684,0.479267,479.266667,26.666667,60.0,826.643912,479.266667,1392.577245
6,S2_A7_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A7,0.000111,0.0005,0.8,0.199389,0.000133,0.168990,...,0.594059,0.96,1216.267579,0.239267,239.266667,26.666667,60.0,1130.826421,239.266667,1456.759754
7,S2_A8_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A8,0.000222,0.0005,0.3,0.699278,0.000267,0.337981,...,0.594059,0.36,456.100342,0.839133,839.133333,53.333333,60.0,343.872471,839.133333,1296.339138
8,S2_A9_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A9,0.000222,0.0005,0.5,0.499278,0.000267,0.337981,...,0.594059,0.60,760.167237,0.599133,599.133333,53.333333,60.0,648.054980,599.133333,1360.521647
9,S2_A10_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2,A10,0.000222,0.0005,0.7,0.299278,0.000267,0.337981,...,0.594059,0.84,1064.234132,0.359133,359.133333,53.333333,60.0,952.237490,359.133333,1424.704157


### Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
         
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.

* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

In [3]:
path = r"C:\Users\Edwin\Downloads\07_08_21_pt2.xlsx"
plate_names = ['Sheet1']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

Should be length of sample_info + 1 (will remove the wavelength column shortly).

In [4]:
merged_df= PlateReader.merge_wavelength_dfs(plate_dfs)

***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [5]:
merged_dfi = PlateReader.detect_ovflw(merged_df, holder = float('nan'))
merged_dfi

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
Wavelength,300.000,305.000,310.000,315.000,320.000,325.000,330.000,335.000,340.000,345.000,...,655.000,660.000,665.000,670.000,675.000,680.000,685.000,690.000,695.000,700.000
D1,0.269,0.193,0.163,0.146,0.127,0.113,0.101,0.087,0.076,0.070,...,0.039,0.039,0.039,0.039,0.039,0.039,0.038,0.039,0.039,0.039
D2,0.277,0.199,0.169,0.151,0.131,0.116,0.104,0.089,0.077,0.071,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
D3,0.280,0.203,0.172,0.154,0.134,0.118,0.106,0.090,0.078,0.071,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
D4,0.352,0.267,0.231,0.210,0.185,0.166,0.151,0.131,0.117,0.107,...,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.041,0.042,0.041
D5,0.661,0.560,0.514,0.481,0.447,0.419,0.395,0.366,0.344,0.327,...,0.097,0.096,0.094,0.093,0.092,0.091,0.090,0.089,0.089,0.088
D6,0.275,0.199,0.168,0.151,0.131,0.116,0.104,0.089,0.078,0.071,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.039,0.040,0.040
D7,0.278,0.202,0.171,0.153,0.134,0.118,0.106,0.091,0.079,0.073,...,0.041,0.041,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
D8,0.677,0.582,0.532,0.498,0.462,0.432,0.407,0.379,0.355,0.336,...,0.081,0.080,0.079,0.078,0.077,0.076,0.075,0.074,0.074,0.073
D9,0.885,0.796,0.754,0.726,0.696,0.671,0.650,0.626,0.606,0.590,...,0.277,0.275,0.272,0.270,0.267,0.264,0.262,0.260,0.257,0.255


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [6]:
platereader_df = PlateReader.rehead_wavelengths(merged_dfi)

platereader_df

,300.0nm,305.0nm,310.0nm,315.0nm,320.0nm,325.0nm,330.0nm,335.0nm,340.0nm,345.0nm,...,655.0nm,660.0nm,665.0nm,670.0nm,675.0nm,680.0nm,685.0nm,690.0nm,695.0nm,700.0nm
D1,0.269,0.193,0.163,0.146,0.127,0.113,0.101,0.087,0.076,0.070,...,0.039,0.039,0.039,0.039,0.039,0.039,0.038,0.039,0.039,0.039
D2,0.277,0.199,0.169,0.151,0.131,0.116,0.104,0.089,0.077,0.071,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
D3,0.280,0.203,0.172,0.154,0.134,0.118,0.106,0.090,0.078,0.071,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
D4,0.352,0.267,0.231,0.210,0.185,0.166,0.151,0.131,0.117,0.107,...,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.041,0.042,0.041
D5,0.661,0.560,0.514,0.481,0.447,0.419,0.395,0.366,0.344,0.327,...,0.097,0.096,0.094,0.093,0.092,0.091,0.090,0.089,0.089,0.088
D6,0.275,0.199,0.168,0.151,0.131,0.116,0.104,0.089,0.078,0.071,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.039,0.040,0.040
D7,0.278,0.202,0.171,0.153,0.134,0.118,0.106,0.091,0.079,0.073,...,0.041,0.041,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
D8,0.677,0.582,0.532,0.498,0.462,0.432,0.407,0.379,0.355,0.336,...,0.081,0.080,0.079,0.078,0.077,0.076,0.075,0.074,0.074,0.073
D9,0.885,0.796,0.754,0.726,0.696,0.671,0.650,0.626,0.606,0.590,...,0.277,0.275,0.272,0.270,0.267,0.264,0.262,0.260,0.257,0.255
D10,0.273,0.197,0.167,0.150,0.130,0.115,0.103,0.088,0.077,0.070,...,0.039,0.039,0.039,0.039,0.039,0.039,0.038,0.038,0.039,0.039


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [7]:
spectras = platereader_df
sample_info.reset_index(drop=True, inplace=True)
spectras.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, spectras], axis = 1)[:len(sample_info)] # switch to the dicionary version since you could store a unblanked, blanked and a blanker in one file.
combined_df

,UID,Labware,Slot,Well,ODE concentration wtf,SDS concentration wtf,ethanol concentration wtf,water concentration wtf,ODE amount mass g,ODE amount volume uL,...,655.0nm,660.0nm,665.0nm,670.0nm,675.0nm,680.0nm,685.0nm,690.0nm,695.0nm,700.0nm
0,S2_A1_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A1,0.000000,0.0005,0.3,0.699500,0.000000,0.000000,...,0.039,0.039,0.039,0.039,0.039,0.039,0.038,0.039,0.039,0.039
1,S2_A2_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A2,0.000000,0.0005,0.5,0.499500,0.000000,0.000000,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
2,S2_A3_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A3,0.000000,0.0005,0.7,0.299500,0.000000,0.000000,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
3,S2_A4_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A4,0.000111,0.0005,0.2,0.799389,0.000133,0.168990,...,0.042,0.042,0.042,0.042,0.042,0.042,0.042,0.041,0.042,0.041
4,S2_A5_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A5,0.000111,0.0005,0.4,0.599389,0.000133,0.168990,...,0.097,0.096,0.094,0.093,0.092,0.091,0.090,0.089,0.089,0.088
5,S2_A6_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A6,0.000111,0.0005,0.6,0.399389,0.000133,0.168990,...,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.039,0.040,0.040
6,S2_A7_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A7,0.000111,0.0005,0.8,0.199389,0.000133,0.168990,...,0.041,0.041,0.040,0.040,0.040,0.040,0.040,0.040,0.040,0.040
7,S2_A8_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A8,0.000222,0.0005,0.3,0.699278,0.000267,0.337981,...,0.081,0.080,0.079,0.078,0.077,0.076,0.075,0.074,0.074,0.073
8,S2_A9_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A9,0.000222,0.0005,0.5,0.499278,0.000267,0.337981,...,0.277,0.275,0.272,0.270,0.267,0.264,0.262,0.260,0.257,0.255
9,S2_A10_07-09-2021,Abgene 96 Well Plate 2200 ÂµL,2.0,A10,0.000222,0.0005,0.7,0.299278,0.000267,0.337981,...,0.039,0.039,0.039,0.039,0.039,0.039,0.038,0.038,0.039,0.039


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [8]:
temp_hold_path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Temp or Working\07_08_21_ODE_SDS_Unblanked_Abs_Pt2"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)